In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import time
import warnings
import logging
import joblib
import shutil

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import shap
import optuna

from datetime import datetime

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score, confusion_matrix,
    precision_recall_curve, roc_curve, auc, make_scorer,
    precision_score, recall_score, balanced_accuracy_score,
    log_loss, matthews_corrcoef, classification_report
)
from sklearn.inspection import permutation_importance

from catboost import CatBoostClassifier

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# ========= CONFIG ===========================================================
RAW_CSV         = '/content/drive/MyDrive/Colab Notebooks/패턴인식/data/train.csv'
BASE_DIR        = '/content/drive/MyDrive/Colab Notebooks/패턴인식'
USE_GPU         = False             # Colab GPU 사용 시 True
GPU_DEVICES     = "0"
GPU_RAM_PART    = 0.50              # GPU 메모리 사용 비율 (CatBoost 전용)
GLOBAL_SEED     = 42

N_TRIALS_OPTUNA = 30                # Optuna 탐색 횟수
SEARCH_N_JOBS   = 1                 # Optuna 병렬 worker 수
TRAIN_ITER      = 1                 # trial-/final-model 공통 반복 횟수
EARLY_STOP      = 50
N_DPND_FEATS    = 3                 # SHAP dependence plot용 상위 피처 개수
# ============================================================================

start_time = time.time()

if USE_GPU:
    os.environ["CUDA_VISIBLE_DEVICES"] = GPU_DEVICES
else:
    os.environ.pop("CUDA_VISIBLE_DEVICES", None)

np.random.seed(GLOBAL_SEED)

# -------- 결과 디렉토리 생성 ------------------------------------------------
log_root   = os.path.join(BASE_DIR, 'log')
timestamp  = datetime.now().strftime('%Y%m%d_%H%M%S')
RES_DIR    = os.path.join(log_root, f'optuna_run_{timestamp}')
PLOT_DIR   = os.path.join(RES_DIR, 'img')

os.makedirs(RES_DIR, exist_ok=True)
os.makedirs(PLOT_DIR, exist_ok=True)

MODEL_CBM  = os.path.join(RES_DIR, 'final_model.cbm')
FI_CSV     = os.path.join(RES_DIR, 'feature_importances.csv')
PI_CSV     = os.path.join(RES_DIR, 'permutation_importances.csv')
LOG_TXT    = os.path.join(RES_DIR, 'log.txt')

logging.root.handlers.clear()

logging.basicConfig(
    level    = logging.INFO,
    format   = "%(asctime)s [%(levelname)s] %(message)s",
    handlers = [logging.FileHandler(LOG_TXT), logging.StreamHandler()]
)
logger = logging.getLogger()
logger.setLevel(logging.INFO)

logger.info("===== 스크립트 시작 =====")
logger.info("CUDA_VISIBLE_DEVICES=%s", os.getenv("CUDA_VISIBLE_DEVICES", "CPU"))


# ============================================================================
# 1. Data Load & Derived Feature 생성
# ============================================================================
logger.info("1) Loading data from: %s", RAW_CSV)
df = pd.read_csv(RAW_CSV)
# 'id', 'shares' 제거, 'y'를 타겟으로 사용
X_full = df.drop(columns=['id', 'shares', 'y'], errors='ignore').copy()
y_full = df['y'].copy()

EPS = 1e-6
def add_derived(df_):
    df_ = df_.copy()
    if {'n_tokens_content','num_imgs'}.issubset(df_.columns):
        df_['feat_content_to_img_ratio'] = df_['n_tokens_content'] / (df_['num_imgs'] + EPS)
    if {'global_subjectivity','global_sentiment_polarity'}.issubset(df_.columns):
        df_['feat_global_sentiment_strength'] = df_['global_subjectivity'] * df_['global_sentiment_polarity']
    if {'n_tokens_content','num_hrefs'}.issubset(df_.columns):
        df_['feat_content_to_href_ratio'] = df_['n_tokens_content'] / (df_['num_hrefs'] + EPS)
    return df_

X_full = add_derived(X_full)

# Train/Validation Hold-out 분할
X_tr, X_val, y_tr, y_val = train_test_split(
    X_full, y_full, test_size=0.2, stratify=y_full, random_state=GLOBAL_SEED
)
logger.info("   - Train shape: %s, Validation shape: %s", X_tr.shape, X_val.shape)

# 결측치 처리: 숫자형 → 중앙값, 범주형 → 'missing'
num_cols = X_tr.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in ['data_channel', 'weekday'] if c in X_tr.columns]

meds = X_tr[num_cols].median()
X_tr[num_cols]  = X_tr[num_cols].fillna(meds)
X_val[num_cols] = X_val[num_cols].fillna(meds)

if cat_cols:
    for c in cat_cols:
        X_tr[c].fillna('missing', inplace=True)
        X_val[c].fillna('missing', inplace=True)
else:
    logger.info("No categorical columns identified for imputation.")

# 클래스 불균형 가중치 계산
pos, neg = (y_tr == 1).sum(), (y_tr == 0).sum()
scale_pos_weight = neg / pos if pos else 1.0
logger.info("scale_pos_weight=%.3f  (neg=%d, pos=%d)", scale_pos_weight, neg, pos)


# ============================================================================
# 2. Custom CompositeMetric for CatBoost
# ============================================================================
class CompositeMetric:
    def get_final_error(self, error, weight):
        return error / (weight + 1e-10)
    def is_max_optimal(self):
        return True
    def evaluate(self, approxes, target, weight):
        prob = 1.0 / (1.0 + np.exp(-approxes[0]))
        pred = (prob >= 0.5).astype(int)
        m = (accuracy_score(target, pred)
             + f1_score(target, pred)
             + roc_auc_score(target, prob)) / 3
        return m, 1.0


# ============================================================================
# 3. Optuna Objective 정의
# ============================================================================
def cb_params(trial):
    return {
        'iterations'           : TRAIN_ITER,
        'learning_rate'        : trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'depth'                : trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg'          : trial.suggest_float('l2_leaf_reg', 1.0, 12.0, log=True),
        'border_count'         : trial.suggest_int('border_count', 32, 254),
        'bagging_temperature'  : trial.suggest_float('bagging_temperature', 0.0, 2.0),
        'random_strength'      : trial.suggest_float('random_strength', 0.1, 5.0, log=True),
        'colsample_bylevel'    : trial.suggest_float('colsample_bylevel', 0.6, 1.0),
        'task_type'            : 'GPU' if USE_GPU else 'CPU',
        'devices'              : GPU_DEVICES if USE_GPU else None,
        'gpu_ram_part'         : GPU_RAM_PART if USE_GPU else None,
        'scale_pos_weight'     : scale_pos_weight,
        'eval_metric'          : CompositeMetric(),
        'early_stopping_rounds': EARLY_STOP,
        'use_best_model'       : True,
        'verbose'              : False,
        'random_state'         : GLOBAL_SEED,
    }

def objective(trial):
    params = cb_params(trial)
    model = CatBoostClassifier(**params)
    model.fit(
        X_tr, y_tr,
        cat_features=cat_cols or None,
        eval_set=[(X_val, y_val)],
        verbose=False
    )
    # 최적 CompositeMetric을 반환
    return model.get_best_score()['validation']['CompositeMetric']

logger.info("Optuna search (%d trials) 시작", N_TRIALS_OPTUNA)
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=N_TRIALS_OPTUNA, n_jobs=SEARCH_N_JOBS, show_progress_bar=True)

best_params = study.best_params
best_score  = study.best_value
logger.info("Optuna best params: %s  |  best Composite=%.4f", best_params, best_score)


# ============================================================================
# 4. Final Model 학습
# ============================================================================
# Optuna에서 찾은 best_params로 학습
final_params = cb_params(optuna.trial.FixedTrial(best_params))
# 최종 학습 시 iterations=TRAIN_ITER, verbose= 3 으로 설정
final_params.update({'iterations': TRAIN_ITER, 'verbose': 3})

final_model = CatBoostClassifier(**final_params)
final_model.fit(
    X_tr, y_tr,
    cat_features=cat_cols or None,
    eval_set=[(X_val, y_val)]
)

best_iter = final_model.get_best_iteration()
logger.info("Best iteration via early-stopping: %d", best_iter)
final_model.save_model(MODEL_CBM)


# ============================================================================
# 5. Learning-Curve Plot (Logloss & Composite Proxy)
# ============================================================================
def save_learning_curve(metric_key, fname):
    ev  = final_model.get_evals_result()
    tr  = ev['learn'][metric_key]
    val = ev['validation'][metric_key]
    plt.figure(figsize=(6,4))
    plt.plot(tr,  label=f'train {metric_key}')
    plt.plot(val, label=f'valid {metric_key}')
    plt.xlabel('Iteration')
    plt.ylabel(metric_key)
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(PLOT_DIR, fname), dpi=300)
    plt.close()

save_learning_curve('Logloss', 'learning_curve_logloss.jpg')
save_learning_curve('CompositeMetric', 'learning_curve_composite.jpg')


# ============================================================================
# 6. Hold-Out Validation 성능 평가
# ============================================================================
logger.info("6) Hold-out Validation Performance 계산 중...")
y_prob = final_model.predict_proba(X_val)[:, 1]
y_pred = (y_prob >= 0.5).astype(int)

# 주요 지표 계산
acc_val            = accuracy_score(y_val, y_pred)
precision_val      = precision_score(y_val, y_pred)
recall_val         = recall_score(y_val, y_pred)
f1_val             = f1_score(y_val, y_pred)
auc_val            = roc_auc_score(y_val, y_prob)
mcc_val            = matthews_corrcoef(y_val, y_pred)
balanced_acc_val   = balanced_accuracy_score(y_val, y_pred)
logloss_val        = log_loss(y_val, y_prob)
comp_val           = (acc_val + f1_val + auc_val) / 3

# Top-K Precision 계산 (상위 10% positive)
k_val              = max(1, int(0.1 * sum(y_val)))
top_k_indices_val  = np.argsort(y_prob)[-k_val:]
top_k_precision_val = sum(y_val.iloc[top_k_indices_val]) / k_val

logger.info("Hold-out Acc=%.4f  Precision=%.4f  Recall=%.4f  F1=%.4f  AUC=%.4f",
            acc_val, precision_val, recall_val, f1_val, auc_val)
logger.info("       MCC=%.4f  BalAcc=%.4f  LogLoss=%.4f  Composite=%.4f  Top-%d Precision=%.4f",
            mcc_val, balanced_acc_val, logloss_val, comp_val, k_val, top_k_precision_val)

# Confusion Matrix (Hold-out)
cm = confusion_matrix(y_val, y_pred)
plt.figure(figsize=(4,3))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Not Popular','Popular'],
            yticklabels=['Not Popular','Popular'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix (Hold-out)')
plt.tight_layout()
plt.savefig(os.path.join(PLOT_DIR, 'confusion_matrix_holdout.jpg'), dpi=300)
plt.close()

# Classification Report (Hold-out)
class_report = classification_report(y_val, y_pred, target_names=['Not Popular','Popular'])
logger.info("\nClassification Report (Hold-out):\n%s", class_report)

# Precision-Recall Curve (Hold-out)
prec_curve, rec_curve, _ = precision_recall_curve(y_val, y_prob)
pr_auc_val               = auc(rec_curve, prec_curve)
plt.figure(figsize=(6,4))
plt.plot(rec_curve, prec_curve, label=f'PR AUC={pr_auc_val:.4f}', lw=2)
plt.axhline(y=sum(y_val)/len(y_val), color='red', lw=1, linestyle='--',
            label=f'Baseline={sum(y_val)/len(y_val):.3f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve (Hold-out)')
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(PLOT_DIR, 'pr_curve_holdout.jpg'), dpi=300)
plt.close()

# ROC Curve (Hold-out)
fpr, tpr, _ = roc_curve(y_val, y_prob)
roc_auc_val = auc(fpr, tpr)
plt.figure(figsize=(6,4))
plt.plot(fpr, tpr, label=f'ROC AUC={roc_auc_val:.4f}', lw=2)
plt.plot([0,1],[0,1],'--', lw=1, color='gray')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve (Hold-out)')
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(PLOT_DIR, 'roc_curve_holdout.jpg'), dpi=300)
plt.close()


# ============================================================================
# 7. 5-Fold Cross-Validation (Manual Loop)
# ============================================================================
logger.info("7) 5-Fold Cross-Validation 시작...")
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=GLOBAL_SEED)

acc_scores, f1_scores, roc_auc_scores = [], [], []
precision_scores, recall_scores = [], []
bal_acc_scores, logloss_scores = [], []
mcc_scores = []
topk_scores = []

# best_iter가 0 이하일 때는 1로 보정
n_iter = best_iter if (best_iter and best_iter > 0) else 1

for fold_id, (train_idx, valid_idx) in enumerate(skf.split(X_tr, y_tr), start=1):
    X_tr_fold, X_va_fold = X_tr.iloc[train_idx], X_tr.iloc[valid_idx]
    y_tr_fold, y_va_fold = y_tr.iloc[train_idx], y_tr.iloc[valid_idx]

    # Fold별 모델 생성 (same best_params + iterations)
    fold_model = CatBoostClassifier(
        iterations=n_iter,
        task_type='GPU' if USE_GPU else 'CPU',
        verbose=0,
        random_state=GLOBAL_SEED,
        scale_pos_weight=scale_pos_weight,
        **best_params
    )

    fold_model.fit(
        X_tr_fold, y_tr_fold,
        cat_features=cat_cols or None,
        eval_set=[(X_va_fold, y_va_fold)],
        verbose=False
    )

    y_va_prob = fold_model.predict_proba(X_va_fold)[:, 1]
    y_va_pred = (y_va_prob >= 0.5).astype(int)

    acc_f        = accuracy_score(y_va_fold, y_va_pred)
    f1_f         = f1_score(y_va_fold, y_va_pred)
    roc_auc_f    = roc_auc_score(y_va_fold, y_va_prob)
    precision_f  = precision_score(y_va_fold, y_va_pred)
    recall_f     = recall_score(y_va_fold, y_va_pred)
    bal_acc_f    = balanced_accuracy_score(y_va_fold, y_va_pred)
    logloss_f    = log_loss(y_va_fold, y_va_prob)
    mcc_f        = matthews_corrcoef(y_va_fold, y_va_pred)

    # Top-K Precision (상위 10% positive)
    k_fold       = max(1, int(0.1 * sum(y_va_fold)))
    topk_idx     = np.argsort(y_va_prob)[-k_fold:]
    topk_prec_f  = sum(y_va_fold.iloc[topk_idx]) / k_fold

    # 리스트에 저장
    acc_scores.append(acc_f)
    f1_scores.append(f1_f)
    roc_auc_scores.append(roc_auc_f)
    precision_scores.append(precision_f)
    recall_scores.append(recall_f)
    bal_acc_scores.append(bal_acc_f)
    logloss_scores.append(logloss_f)
    mcc_scores.append(mcc_f)
    topk_scores.append(topk_prec_f)

    # Fold별 로그 출력
    logger.info(
        "Fold %d ➔ Acc=%.4f  F1=%.4f  ROC_AUC=%.4f  Prec=%.4f  Recall=%.4f  BalAcc=%.4f  MCC=%.4f  LogLoss=%.4f  Top-%d_Prec=%.4f",
        fold_id, acc_f, f1_f, roc_auc_f, precision_f, recall_f, bal_acc_f, mcc_f, logloss_f, k_fold, topk_prec_f
    )

# 평균 및 표준편차 계산 함수
def mean_std(lst):
    return np.mean(lst), np.std(lst)

acc_mean,    acc_std    = mean_std(acc_scores)
f1_mean,     f1_std     = mean_std(f1_scores)
roc_mean,    roc_std    = mean_std(roc_auc_scores)
prec_mean,   prec_std   = mean_std(precision_scores)
rec_mean,    rec_std    = mean_std(recall_scores)
bal_mean,    bal_std    = mean_std(bal_acc_scores)
ll_mean,     ll_std     = mean_std(logloss_scores)
mcc_mean,    mcc_std    = mean_std(mcc_scores)
topk_mean,   topk_std   = mean_std(topk_scores)

logger.info("\n=== 5-Fold CV Results (Mean ± 2*STD) ===")
logger.info("Accuracy:          %.4f (+/- %.4f)", acc_mean,    acc_std * 2)
logger.info("F1 Score:          %.4f (+/- %.4f)", f1_mean,     f1_std * 2)
logger.info("ROC AUC:           %.4f (+/- %.4f)", roc_mean,    roc_std * 2)
logger.info("Precision:         %.4f (+/- %.4f)", prec_mean,   prec_std * 2)
logger.info("Recall:            %.4f (+/- %.4f)", rec_mean,    rec_std * 2)
logger.info("Balanced Accuracy: %.4f (+/- %.4f)", bal_mean,    bal_std * 2)
logger.info("Log Loss:          %.4f (+/- %.4f)", ll_mean,     ll_std * 2)
logger.info("MCC:               %.4f (+/- %.4f)", mcc_mean,    mcc_std * 2)
logger.info("Top-10%% Precision: %.4f (+/- %.4f)", topk_mean,   topk_std * 2)


# ============================================================================
# 8. Feature Importance & Permutation Importance
# ============================================================================
logger.info("8) Feature Importance 저장 중...")
# 내장된 Feature Importance (prettified=True)
fi = final_model.get_feature_importance(prettified=True)
fi.to_csv(FI_CSV, index=False)

logger.info("   Permutation Importance 계산 중...")
def composite_scorer_func(y_true, y_pred_proba, **kw):
    prob = y_pred_proba[:, 1] if y_pred_proba.ndim == 2 else y_pred_proba
    pred = (prob >= 0.5).astype(int)
    return (accuracy_score(y_true, pred)
            + f1_score(y_true, pred)
            + roc_auc_score(y_true, prob)) / 3

custom_scorer = make_scorer(composite_scorer_func, needs_proba=True)

pi = permutation_importance(
    final_model, X_val, y_val,
    scoring=custom_scorer,
    n_repeats=5, random_state=GLOBAL_SEED, n_jobs=-1
)
pi_df = pd.DataFrame({
    'feature'        : X_val.columns,
    'importance_mean': pi.importances_mean,
    'importance_std' : pi.importances_std
})
pi_df.to_csv(PI_CSV, index=False)


# ============================================================================
# 9. SHAP Summary & Dependence Plot
# ============================================================================
logger.info("9) SHAP 계산 중 (시간 소요될 수 있음)...")
explainer = shap.TreeExplainer(final_model)
shap_vals = explainer.shap_values(X_val, check_additivity=False)

# SHAP Summary Plot
plt.figure(figsize=(8,6))
shap.summary_plot(shap_vals, X_val, show=False)
plt.tight_layout()
plt.savefig(os.path.join(PLOT_DIR, 'shap_summary.jpg'), dpi=300)
plt.close()

# SHAP Dependence Plot: 'Feature Id' 컬럼에서 상위 N_DPND_FEATS개를 추출
top_feats = fi.sort_values('Importances', ascending=False)['Feature Id'].head(N_DPND_FEATS)
for feat in top_feats:
    plt.figure(figsize=(6,4))
    shap.dependence_plot(feat, shap_vals, X_val,
                         interaction_index='auto', show=False)
    fn = f"shap_dependence_{feat.replace('/','_')}.jpg"
    plt.tight_layout()
    plt.savefig(os.path.join(PLOT_DIR, fn), dpi=300)
    plt.close()


# ============================================================================
# 10. 로그 요약 및 완료
# ============================================================================
elapsed = time.time() - start_time if 'start_time' in globals() else 0
h, m, s = int(elapsed // 3600), int((elapsed % 3600) // 60), int(elapsed % 60)
logger.info("Total runtime: %dh %dm %ds", h, m, s)
logger.info("Artifacts saved to %s", RES_DIR)